In [1]:
import requests, folium, pandas as pd
from folium.plugins import FastMarkerCluster, MarkerCluster

import json
import urllib.request
import datetime
import webbrowser

from matplotlib import font_manager, rc

font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)

In [3]:
def get_loc(query):
    key = "" # kakao api key 

    name, x, y = [], [], []

    for page in range(1,7):
        # url for getting all the subway location in [query] region 
        url2 = "https://dapi.kakao.com/v2/local/search/keyword.json?query={}&category_group_code=SW8&page={}".format(query, page)
        headers = {'Authorization': 'KakaoAK {0}'.format(key)}
        res = requests.get(url2, headers=headers)
        res = res.json()
        res = res['documents']
        
        # make a list of [station name, x(longitude), y(latitude)]
        for i in res:
            if i['place_name'] not in name:
                name.append(i['place_name'])
                x.append(i['x'])
                y.append(i['y'])

    print(len(name))
    return name, x, y

In [4]:
def marker_cafe(loc_y,loc_x, query):
    key = "" #kakao api key
    # url for getting location of every [query] store within a circle of radius 1000m
    url = "https://dapi.kakao.com/v2/local/search/keyword.json?y={}&x={}&radius=1000&query={}".format(loc_y,loc_x,query)
    headers = {'Authorization': 'KakaoAK {0}'.format(key)}
    res = requests.get(url, headers=headers).json()
    name, x, y = [], [], []
    
    # make a list of [station name, x(longitude), y(latitude)]
    for i in res['documents']:
        name.append(i['place_name'])
        x.append(i['x'])
        y.append(i['y'])
    
    # adding markers of the group of stores at the map.  
        cafe_df = pd.DataFrame({"name" : name, "x" : x, "y" : y})
        for row in cafe_df.iterrows():
            folium.Marker([row[1]['y'],row[1]['x']], popup=row[1]['name']).add_to(map_osm)

        map_osm.save('index.html')
        map_osm


In [5]:
name, x, y = [], [], []
for i in ['대구 1호선', '대구 2호선', '대구 3호선', '경산']:
    name1, x1, y1 = get_loc(i) # get location of regions above
    name += name1
    x += x1
    y += y1

32
26
30
3


In [6]:
station_df = pd.DataFrame(data=name,)
station_df['x'] = x
station_df['y'] = y

In [7]:
station_df.rename(columns={0:'name'},inplace=True)

In [8]:
station_df.head(3) 

,name,x,y
0,반월당역 대구1호선,128.593337428631,35.8645000309428
1,안심역 대구1호선,128.733769742803,35.8712130165849
2,상인역 대구1호선,128.537763550346,35.8189266589744


In [9]:
station_df_sort = station_df.sort_values(by = ['name'], axis=0)

In [10]:
key = "" # kakao api key
query = "스타벅스"

# drawing of a circle of 1000m radius within stations 
map_osm = folium.Map(location=[station_df['y'][0], station_df['x'][0]], zoom_start=12)
folium.Circle([station_df['y'][0], station_df['x'][0]], radius=1000, 
                    popup=station_df['name'][0], color='#3186cc',
                    fill_color='#3186cc').add_to(map_osm) # station name: 반월당역

folium.Circle([station_df['y'][13], station_df['x'][13]], radius=1000, 
                    popup=station_df['name'][13], color='#3186cc',
                    fill_color='#3186cc').add_to(map_osm) # station name: 동대구역

folium.Circle([station_df['y'][2], station_df['x'][2]], radius=1000, 
                    popup=station_df['name'][2], color='#3186cc',
                    fill_color='#3186cc').add_to(map_osm) # station name: 상인역

folium.Circle([station_df['y'][37], station_df['x'][37]], radius=1000, 
                    popup=station_df['name'][2], color='#3186cc',
                    fill_color='#3186cc').add_to(map_osm) # station name: 범어역

# adding markers of starbucks within 1000m of each station 
# 반월당역
marker_cafe(station_df['y'][0],station_df['x'][0],query)
# starbucks at 상인역
marker_cafe(station_df['y'][2],station_df['x'][2],query)
# starbucks at 동대구역
marker_cafe(station_df['y'][13],station_df['x'][13],query)
# starbucks at 범어역 
marker_cafe(station_df['y'][37],station_df['x'][37],query)


In [11]:
map_osm

In [13]:
sub_star = {'station': '','star':[]}
sub_star
sub_star_list=[]

for i in range(0,len(station_df)):
    sub_star['station']=[station_df['name'][i],station_df['y'][i],station_df['x'][i]]
    y=sub_star['station'][1]
    x=sub_star['station'][2]
    
    query="스타벅스"
    url = "https://dapi.kakao.com/v2/local/search/keyword.json?y={}&x={}&radius=1000&query={}".format(y,x,query)
    headers = {'Authorization': 'KakaoAK {0}'.format(key)}
    res = requests.get(url, headers=headers).json()

    for j in res['documents']:
        if j['place_name'] not in name:
            sub_star['star'].append([j['place_name'],j['y'],j['x']])
    sub_star_list.append({'station':sub_star['station'],'star':sub_star['star']})
    sub_star['star']=[]
    

In [15]:
#그룹 나누기 (6개 이상/4개이상/1개이상)
dic = {'station': '','star':[]}
firstg = []
secondg = []
thirdg = []

for i in range(0,len(sub_star_list)):
    if len(sub_star_list[i]['star']) >= 6:
        dic['station']=sub_star_list[i]['station']
        dic['star'].append(sub_star_list[i]['star'])
        firstg.append(dic)
        dic={'station': '','star':[]}
    elif len(sub_star_list[i]['star']) >= 4:
        dic['station']=sub_star_list[i]['station']
        dic['star'].append(sub_star_list[i]['star'])
        secondg.append(dic)
        dic={'station': '','star':[]}
    elif len(sub_star_list[i]['star']) >= 1:
        dic['station']=sub_star_list[i]['station']
        dic['star'].append(sub_star_list[i]['star'])
        thirdg.append(dic)
        dic={'station': '','star':[]}
        

In [18]:
#Circle - station 
#그룹별 지도 써클 및 마커 표시

#첫번째 그룹

m = folium.Map(location=[station_df['y'][0], station_df['x'][0]], zoom_start=12)
for i in range(0,len(thirdg)):
    folium.Circle([thirdg[i]['station'][1], thirdg[i]['station'][2]], radius=1000, 
                        popup=thirdg[i]['station'][0], color='mediumturquoise',
                        fill_color='steelblue').add_to(m) #반월당
    #marker
    for j in range(len(thirdg[i]['star'][0])):
        x = thirdg[i]['star'][0][j][2]
        y = thirdg[i]['star'][0][j][1]
        name = str(thirdg[i]['star'][0][j][0])
        folium.Marker([y,x],popup=name,
                     icon=folium.Icon(color='blue',icon="flag")).add_to(m)

    
for i in range(0,len(secondg)):
    folium.Circle([secondg[i]['station'][1], secondg[i]['station'][2]], radius=1000, 
                        popup=secondg[i]['station'][0], color='mediumpurple',
                        fill_color='indigo').add_to(m) #반월당
    #marker
    for j in range(len(secondg[i]['star'][0])):
        x = secondg[i]['star'][0][j][2]
        y = secondg[i]['star'][0][j][1]
        name = str(secondg[i]['star'][0][j][0])
        folium.Marker([y,x],popup=name,
                     icon=folium.Icon(color='purple',icon="flag")).add_to(m)
        
for i in range(0,len(firstg)):
    folium.Circle([firstg[i]['station'][1], firstg[i]['station'][2]], radius=1000, 
                        popup=firstg[i]['station'][0], color='crimson',
                        fill_color='crimson').add_to(m) #반월당
    #marker
    for j in range(len(firstg[i]['star'][0])):
        x = firstg[i]['star'][0][j][2]
        y = firstg[i]['star'][0][j][1]
        name = str(firstg[i]['star'][0][j][0])
        folium.Marker([y,x],popup=name,
                     icon=folium.Icon(color='red',icon="flag")).add_to(m)
        
m.save('index.html')
m